In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [4]:
crypto_df = pd.read_excel('Investments Tracker.xlsx', sheet_name="Crypto", date_parser="Transaction Date")

In [5]:
crypto_df.head()

,Transaction Date,Platform,Company,Ticker,Transaction,Quantity,Price Per,Total Price,Fees,Notes
0,2017-12-24,Coinbase,Litecoin,LTC,Buy,0.8338,230.6800,-200.0000,7.6700,NaN
1,2017-12-30,Coinbase,Litecoin,LTC,Buy,0.2041,186.2466,-38.0100,1.9900,NaN
2,2021-02-02,Kraken,Dogecoin,XDG,Buy,3553.0300,0.0281,-100.0000,nan,NaN
3,2021-02-04,Kraken,Dogecoin,XDG,Sell,-3553.0300,0.0414,147.0700,nan,NaN
4,2021-02-07,Kraken,Dogecoin,XDG,Buy,1699.7000,0.0588,-100.0000,nan,NaN


# Crypto

In [ ]:
print("Coin balance")
crypto_df.groupby("Company")["Quantity"].sum()

In [ ]:
print("Total Spent on each ")
crypto_df.groupby("Company")["Total Price"].sum()

In [ ]:
print("Average cost base analysis")
coins_total = crypto_df.groupby("Company").agg({"Quantity": np.sum, "Total Price": np.sum}).reset_index()
coins_total["Avg Cost"] = abs(coins_total["Total Price"]) / coins_total["Quantity"]
coins_total

In [ ]:
def coins_sell_break_even(total_value, current_price):
    price = abs(total_value)
    return price / current_price

def coins_current_value(total_coins, current_price):
    return total_coins * current_price

def coins_sell_all_profit(total_coins, total_price, current_price):
    return total_coins * current_price - abs(total_price)

In [ ]:
xgd_price = 0.0442501
xgd_data = coins_total.loc["Dogecoin"]
print(coins_sell_break_even(xgd_data.Price, xgd_price))
print(coins_current_value(xgd_data.Coins, xgd_price))
print(coins_sell_all_profit(xgd_data.Coins, xgd_data.Price, xgd_price))

# Stonks

In [6]:
def stonk_split(df, company, date, ratio):
    co_filter = (df["Company"] == company) | (df["Ticker"] == company)
    date_filter = (df["Transaction Date"] <= date)
    
    if ratio == "4:1":
        df.loc[(co_filter & date_filter), "Quantity"] *= 4
        df.loc[(co_filter & date_filter), "Price Per"] /= 4
    return df

In [7]:
stonks_df = pd.read_excel('Investments Tracker.xlsx', sheet_name="Stonks", date_parser="Transaction Date")
stonks_df = stonk_split(stonks_df, "AAPL", "2020-08-28", "4:1")
stonks_df.head()

,Transaction Date,Platform,Company,Ticker,Transaction,Quantity,Price Per,Total Price,Fees,Notes
0,2020-08-06,Robinhood,Marathon Oil,MRO,Bonus Stock,1.0000,5.8200,0.0000,nan,NaN
1,2020-08-06,Robinhood,Twitter,TWTR,Buy,1.0027,36.9000,-37.0000,nan,NaN
2,2020-08-06,Robinhood,Nikola,NKLA,Buy,0.3714,35.0000,-13.0000,nan,NaN
3,2020-08-08,Robinhood,Apple,AAPL,Buy,1.7775,112.5175,-199.9999,nan,stock split 4:1
4,2020-08-12,Robinhood,Dynavax,DVAX,Buy,2.7630,7.2400,-20.0038,nan,NaN


In [ ]:
print("Stonks balance")
stonks_df.groupby("Company")["Quantity"].sum()

In [ ]:
print("Total Spent on each ")
stonks_df.groupby("Company")["Total Price"].sum()

In [ ]:
print("Profit and Loss")
sold_stonks = (stonks_df
                .groupby("Company")
                .filter(lambda x: x["Quantity"].sum().round(0) < 1)
                .groupby("Company")
                .agg({"Quantity": np.sum, "Total Price": np.sum})
               )
sold_stonks

In [ ]:
print("Average cost base analysis")
stonks_total = (stonks_df
                .groupby(["Company", "Ticker"])
                .filter(lambda x: x["Quantity"].sum() >= 1)
                .groupby(["Company", "Ticker"])
                .agg({"Quantity": np.sum, "Total Price": np.sum})
               )
stonks_total["Avg Cost"] = abs(stonks_total["Total Price"]) / stonks_total["Quantity"]
stonks_total

In [ ]:
# combine

In [37]:
def transform_daily(df):
    df = (df.groupby(["Transaction Date", "Transaction", "Investment Type"])["Total Price"]
          .sum()
          .reset_index()
         )
    df["Net Spend"] = df["Total Price"].apply(lambda x: round(abs(x), 2))
    return df


def stonk_split(df, company, date, ratio):
    co_filter = (df["Company"] == company) | (df["Ticker"] == company)
    date_filter = (df["Transaction Date"] <= date)

    if ratio == "4:1":
        df.loc[(co_filter & date_filter), "Quantity"] *= 4
        df.loc[(co_filter & date_filter), "Price Per"] /= 4
    return df

In [29]:
crypto_df = pd.read_excel('Investments Tracker.xlsx', sheet_name="Crypto", date_parser="Transaction Date")
crypto_df["Investment Type"] = "Crypto"
stonks_df = pd.read_excel('Investments Tracker.xlsx', sheet_name="Stonks", date_parser="Transaction Date")
stonks_df["Investment Type"] = "Stonks"
stonks_df = stonk_split(stonks_df, "AAPL", "2020-08-28", "4:1")
df = pd.concat([crypto_df, stonks_df])

df["Net Spend"] = df['Total Price'].apply(lambda x: round(x, 2) * -1)

# daily spend
investments_daily = (df.groupby(["Transaction Date", "Transaction", "Investment Type"])["Total Price"]
                     .sum()
                     .reset_index()
                    )
investments_daily["Net Spend"] = investments_daily["Total Price"].apply(lambda x: round(abs(x), 2))

# profit and loss
p_and_l = (df
           .groupby(["Company", "Ticker", "Investment Type"])
           .filter(lambda x: x["Quantity"].sum().round(3) == 0)
           .groupby(["Company", "Investment Type"])
           .agg({"Quantity": np.sum, "Total Price": np.sum})
           .reset_index()
           )

# avg cost per asset
investments_total = (df
                     .groupby(["Company", "Ticker", "Investment Type"])
                     .filter(lambda x: x["Quantity"].sum().round(3) > 0)
                     .groupby(["Company", "Ticker", "Investment Type"])
                     .agg({"Quantity": np.sum, "Total Price": np.sum})
                     .reset_index()
                    )
investments_total["Avg Cost"] = abs(investments_total["Total Price"]) / investments_total["Quantity"]

df.head()

,Transaction Date,Platform,Company,Ticker,Transaction,Quantity,Price Per,Total Price,Fees,Notes,Investment Type,Net Spend
0,2017-12-24,Coinbase,Litecoin,LTC,Buy,0.8338,230.6800,-200.0000,7.6700,NaN,Crypto,200.0000
1,2017-12-30,Coinbase,Litecoin,LTC,Buy,0.2041,186.2466,-38.0100,1.9900,NaN,Crypto,38.0100
2,2021-02-02,Kraken,Dogecoin,XDG,Buy,3553.0300,0.0281,-100.0000,nan,NaN,Crypto,100.0000
3,2021-02-04,Kraken,Dogecoin,XDG,Sell,-3553.0300,0.0414,147.0700,nan,NaN,Crypto,-147.0700
4,2021-02-07,Kraken,Dogecoin,XDG,Buy,1699.7000,0.0588,-100.0000,nan,NaN,Crypto,100.0000


In [39]:
# daily spend
investments_daily = transform_daily(df)
investments_daily.head()

,Transaction Date,Transaction,Investment Type,Total Price,Net Spend
0,2017-12-24,Buy,Crypto,-200.0000,200.0000
1,2017-12-30,Buy,Crypto,-38.0100,38.0100
2,2020-08-06,Bonus Stock,Stonks,0.0000,0.0000
3,2020-08-06,Buy,Stonks,-50.0000,50.0000
4,2020-08-08,Buy,Stonks,-199.9999,200.0000


In [41]:
# profit and loss
p_and_l = (df
           .groupby(["Company", "Ticker"])
           .filter(lambda x: x["Quantity"].sum().round(3) == 0)
           .groupby("Company")
           .agg({"Quantity": np.sum, "Total Price": np.sum})
           .reset_index()
           )

p_and_l

,Company,Quantity,Total Price
0,Churchill Capital,0.0000,271.4396
1,Dynavax,0.0000,-5.4983
2,Marathon Oil,0.0000,6.7100
3,Nikola,0.0000,-22.5302
4,Twitter,0.0000,6.5778


In [ ]:
# Crypto
# read and transform data

# coins reduced
coins_reduced = coins_total.drop('Total Price', axis=1)

# daily spend

# Stonks
# read and transform data

# profit and loss


# daily spend

# avg cost per cost

In [14]:
stonks_total

,,Quantity,Total Price,Avg Cost
Company,Ticker,,,
AMMO,POWW,20.0000,-145.6000,7.2800
Aeterna Zentaris,AEZS,50.0000,-140.4300,2.8086
Agenus,Agen,1.0000,0.0000,0.0000
American Airlines,AAL,8.0000,-182.3200,22.7900
Apple,AAPL,7.0000,-830.4691,118.6384
Asia Broadband,AABB,19.0000,-6.0230,0.3170
Atosa,ATOS,4.0000,-13.2585,3.3146
Blink Charging,BLNK,21.0000,-811.8992,38.6619
Exela,XELA,4.0000,-3.3200,0.8300


In [44]:
investments_total = (df
                     .groupby(["Company", "Ticker", "Investment Type"])
                     .filter(lambda x: x["Quantity"].sum().round(3) > 0)
                     .groupby(["Company", "Ticker", "Investment Type"])
                     .agg({"Quantity": np.sum, "Total Price": np.sum})
                     .reset_index()
                    )
investments_total["Avg Cost"] = abs(investments_total["Total Price"]) / investments_total["Quantity"]

investments_total.head()

,Company,Ticker,Investment Type,Quantity,Total Price,Avg Cost
0,AMMO,POWW,Stonks,20.0000,-145.6000,7.2800
1,Aeterna Zentaris,AEZS,Stonks,50.0000,-140.4300,2.8086
2,Agenus,Agen,Stonks,1.0000,0.0000,0.0000
3,American Airlines,AAL,Stonks,8.0000,-182.3200,22.7900
4,Apple,AAPL,Stonks,7.0000,-830.4691,118.6384


In [46]:
investments_total.columns.to_list()

['Company', 'Ticker', 'Investment Type', 'Quantity', 'Total Price', 'Avg Cost']